<a href="https://colab.research.google.com/github/AdityaAnam/2PNC/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
!pip install -U sentence-transformers
!pip install googletrans==3.1.0a0
!pip install arabic-reshaper
!pip install python-bidi
!pip install wordcloud

In [21]:
!pip install pyarabic

In [22]:
import os
#print(os.getcwd())
#print(os.listdir())

for root,dirs,files in os.walk("drive/MyDrive/Data_hadith/"):
        print (root)
        print("FILES:")
        for filename in files:
            print (filename)
            print(os.path.join(root, filename))
            print ('-----------------------')

drive/MyDrive/Data_hadith/
FILES:
all_hadiths_clean.csv
drive/MyDrive/Data_hadith/all_hadiths_clean.csv
-----------------------
data (1).db
drive/MyDrive/Data_hadith/data (1).db
-----------------------


In [71]:
import pandas as pd
import numpy as np
import re
import arabic_reshaper
from bidi.algorithm import get_display
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import pprint
from wordcloud import WordCloud , ImageColorGenerator
from numpy.linalg import norm
from googletrans import Translator
import pyarabic.araby as araby
from sentence_transformers import SentenceTransformer
data= pd.read_csv('drive/MyDrive/Data_hadith/all_hadiths_clean.csv')
data.head()

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en
0,0,1,Sahih Bukhari,1,1,Revelation - كتاب بدء الوحى,"30418, 20005, 11062, 11213, 11042, 3",حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated 'Umar bin Al-Khattab: ...
1,1,2,Sahih Bukhari,1,2,Revelation - كتاب بدء الوحى,"30355, 20001, 11065, 10511, 53",حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن هش...,Narrated 'Aisha: ...
2,2,3,Sahih Bukhari,1,3,Revelation - كتاب بدء الوحى,"30399, 20023, 11207, 11013, 10511, 53",حدثنا يحيى بن بكير، قال حدثنا الليث، عن عقيل، ...,Narrated 'Aisha: (the m...
3,3,4,Sahih Bukhari,1,4,Revelation - كتاب بدء الوحى,"11013, 10567, 34",قال ابن شهاب وأخبرني أبو سلمة بن عبد الرحمن، أ...,Narrated Jabir bin 'Abdullah Al-Ansari while ...
4,4,5,Sahih Bukhari,1,5,Revelation - كتاب بدء الوحى,"20040, 20469, 11399, 11050, 17",حدثنا موسى بن إسماعيل، قال حدثنا أبو عوانة، قا...,Narrated Said bin Jubair: ...


In [24]:
data.text_ar[0]

'حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب  رضى الله عنه  على المنبر قال سمعت رسول الله صلى الله عليه وسلم يقول \u200f"\u200f إنما الأعمال بالنيات، وإنما لكل امرئ ما نوى، فمن كانت هجرته إلى دنيا يصيبها أو إلى امرأة ينكحها فهجرته إلى ما هاجر إليه \u200f"\u200f\u200f.\u200f'

In [25]:
import sqlite3
!pip install setuptools

In [26]:
def splilt_hadith_book(data):
    """
        I will check if the hadith contain the exact search words
        if it conain:
            split it
        else:
            put whole hadith in sanad and matn
    """
    splited_hadiths = []
    searching_word_1 = " رسول الله "
    searching_word_2 = " النبى "
    padding_word = " صلى الله عليه وسلم "
    f1, f2 = 0 , 0
    n_not = 0

    for hadith in data:
        st = ""
        st_1 = re.search(searching_word_1, hadith)
        st_2 = re.search(searching_word_2, hadith)
        # if hadith didn't containt specific spliting word/s
        if st_1 == None and st_2 == None:
            n_not +=1
            splited_hadiths.append({"sanad" : hadith,
                               "matn": hadith})
            continue

         # take the not None
        if st_1 == None:
            st = st_2 
            f2 = 1
        else:
            st = st_1
            f1 = 1
        sanad = hadith[ : st.start()] + (searching_word_2 if f2 else searching_word_1)  + padding_word

        matn = hadith[st.end()-1 + len(padding_word) : ]

        splited_hadiths.append({"sanad" : sanad,
                               "matn": matn})
    return splited_hadiths

In [27]:
def normalize(sentence):
    """
        replace all  صلي or النبي with صلى or النبى
    """
    sentence = re.sub("صلي", "صلى", sentence)
    sentence = re.sub("النبي", "النبى", sentence)
    return sentence

test_st = "قال رسول الله صلي الله عليه وسلم ، قال النبي صلى الله عليه وسلم"
print(test_st)
print(normalize(test_st))

قال رسول الله صلي الله عليه وسلم ، قال النبي صلى الله عليه وسلم
قال رسول الله صلى الله عليه وسلم ، قال النبى صلى الله عليه وسلم


In [31]:
List= []
for hadith in data_test['text_ar']:
    List.append(normalize(hadith))
normalize_data_test= pd.DataFrame(List)
normalize_data_test

,0
0,حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...
1,حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن هش...
2,حدثنا يحيى بن بكير، قال حدثنا الليث، عن عقيل، ...
3,قال ابن شهاب وأخبرني أبو سلمة بن عبد الرحمن، أ...
4,حدثنا موسى بن إسماعيل، قال حدثنا أبو عوانة، قا...
...,...
95,حدثنا مسدد، قال حدثنا أبو عوانة، عن أبي بشر، ع...
96,أخبرنا محمد هو ابن سلام حدثنا المحاربي، قال ...
97,حدثنا سليمان بن حرب، قال حدثنا شعبة، عن أيوب، ...
98,حدثنا عبد العزيز بن عبد الله، قال حدثني سليمان...


In [32]:
List2 = splilt_hadith_book(List)
splitted_data_test= pd.DataFrame(List2)
splitted_data_test['sanad'][0]

'حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب  رضى الله عنه  على المنبر قال سمعت رسول الله  صلى الله عليه وسلم '

# New Section

In [33]:
data_test=data[:100]
data_test

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en
0,0,1,Sahih Bukhari,1,1,Revelation - كتاب بدء الوحى,"30418, 20005, 11062, 11213, 11042, 3",حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated 'Umar bin Al-Khattab: ...
1,1,2,Sahih Bukhari,1,2,Revelation - كتاب بدء الوحى,"30355, 20001, 11065, 10511, 53",حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن هش...,Narrated 'Aisha: ...
2,2,3,Sahih Bukhari,1,3,Revelation - كتاب بدء الوحى,"30399, 20023, 11207, 11013, 10511, 53",حدثنا يحيى بن بكير، قال حدثنا الليث، عن عقيل، ...,Narrated 'Aisha: (the m...
3,3,4,Sahih Bukhari,1,4,Revelation - كتاب بدء الوحى,"11013, 10567, 34",قال ابن شهاب وأخبرني أبو سلمة بن عبد الرحمن، أ...,Narrated Jabir bin 'Abdullah Al-Ansari while ...
4,4,5,Sahih Bukhari,1,5,Revelation - كتاب بدء الوحى,"20040, 20469, 11399, 11050, 17",حدثنا موسى بن إسماعيل، قال حدثنا أبو عوانة، قا...,Narrated Said bin Jubair: ...
...,...,...,...,...,...,...,...,...,...
95,37,96,Sahih Bukhari,3,96,Knowledge - كتاب العلم,"30175, 20469, 11419, 11264, 29",حدثنا مسدد، قال حدثنا أبو عوانة، عن أبي بشر، ع...,Narrated `Abdullah bin `Amr: ...
96,38,97,Sahih Bukhari,3,97,Knowledge - كتاب العلم,"30376, 20333, 15239, 11052, 11901, 41",أخبرنا محمد هو ابن سلام حدثنا المحاربي، قال ...,Narrated Abu Burda's father: ...
97,39,98,Sahih Bukhari,3,98,Knowledge - كتاب العلم,"20193, 20020, 11015, 11029, 17",حدثنا سليمان بن حرب، قال حدثنا شعبة، عن أيوب، ...,Narrated Ibn 'Abbas: ...
98,40,99,Sahih Bukhari,3,99,Knowledge - كتاب العلم,"30105, 20160, 11209, 11168, 13",حدثنا عبد العزيز بن عبد الله، قال حدثني سليمان...,Narrated Abu Huraira: I s...


In [34]:
SplitDT= pd.DataFrame(List2)
type(List2)

list

In [35]:
SplitDT.sanad[0]

'حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب  رضى الله عنه  على المنبر قال سمعت رسول الله  صلى الله عليه وسلم '

In [36]:
import pyarabic.araby as araby


In [37]:
!pip install pyarabic

In [38]:
stop_words =['من','في', 'على','و','فى','يا','عن','مع','ان','هو','علي','ما','اللي','كل','بعد','ده','اليوم','أن','يوم','انا','إلى','كان','ايه','اللى','الى','دي','بين','انت','أنا','حتى','لما','فيه','هذا','واحد','احنا','اي','كده','إن','او','أو','عليه','ف','دى','مين','الي','كانت','أمام','زي','يكون','خلال','ع','كنت','هي','فيها','عند','التي','الذي','قال','هذه','قد','انه','ريتويت','بعض','أول','ايه','الان','أي','منذ','عليها','له','ال','تم','ب','دة','عليك','اى','كلها','اللتى','هى','دا','انك','وهو','ومن','منك','نحن','زى','أنت','انهم','معانا','حتي','وانا','عنه','إلي','ونحن','وانت','منكم','وان','معاهم','معايا','وأنا','عنها','إنه','اني','معك','اننا','فيهم','د','انتا','عنك','وهى','معا','آن','انتي','وأنت','وإن','ومع','وعن','معاكم','معاكو','معاها','وعليه','وانتم','وانتي','¿','|']

In [39]:
def seperate_names(sentence):

    sentence = re.sub("حدثنا", "", sentence)
    sentence = re.sub("قال", "", sentence)
    sentence = re.sub("،", " : ", sentence)
    sentence = re.sub("أنه سمع", "", sentence)
    sentence = re.sub("أخبرني", "", sentence)
    sentence = re.sub("يقول", "", sentence)
    return sentence

test_st = "حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب  رضى الله عنه  على المنبر قال سمعت رسول الله  صلى الله عليه وسلم"
print(seperate_names(test_st))

 الحميدي عبد الله بن الزبير :    سفيان :    يحيى بن سعيد الأنصاري :    محمد بن إبراهيم التيمي :   علقمة بن وقاص الليثي :   سمعت عمر بن الخطاب  رضى الله عنه  على المنبر  سمعت رسول الله  صلى الله عليه وسلم


In [40]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

#Our sentences we like to encode
sentences = data.text_en[:3]
#Sentences are encoded by calling model.encode()
#for sent in sentences:
embeddings = model.encode(sentences,convert_to_numpy=True,normalize_embeddings= True)

#Print the embeddings
#for sentence, embedding in zip(sentences, embeddings):
#   print("Sentence:", sentence)
#    print("Embedding:", embedding)
print(np.dot(embeddings[1],embeddings[0]))
#print(np.dot(embeddings[0],embeddings[1])/(norm(embeddings[0])*norm(embeddings[1])))

0.6433797


In [41]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [42]:
def data_embeddor (data):
  sentences = data.text_en
  embeddings = model.encode(sentences,show_progress_bar=True,normalize_embeddings= True)
  return(embeddings)

In [59]:
def sentence_embeddor (sentence) :
  return (model.encode(sentence,normalize_embeddings= True))

In [55]:
def tran(sentence):
  trans_sentence=[]
  translator = Translator()
  trans_sentence= translator.translate(sentence, dest= 'en')
  return(trans_sentence)

In [43]:
data.text_ar[1]

'حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن هشام بن عروة، عن أبيه، عن عائشة أم المؤمنين  رضى الله عنها  أن الحارث بن هشام  رضى الله عنه  سأل رسول الله صلى الله عليه وسلم فقال يا رسول الله كيف يأتيك الوحى فقال رسول الله صلى الله عليه وسلم \u200f"\u200f أحيانا يأتيني مثل صلصلة الجرس  وهو أشده على  فيفصم عني وقد وعيت عنه ما قال، وأحيانا يتمثل لي الملك رجلا فيكلمني فأعي ما يقول \u200f"\u200f\u200f.\u200f قالت عائشة رضى الله عنها ولقد رأيته ينزل عليه الوحى في اليوم الشديد البرد، فيفصم عنه وإن جبينه ليتفصد عرقا\u200f.\u200f'

In [68]:
A = tran(data.text_ar[1])
for i in range(0,100):
  K=[]
  K.append(np.dot(sentence_embeddor(A.text),sentence_embeddor(data.text_en[i])))
  print(K)

0 [0.694684]
1 [0.72115254]
2 [0.68113375]
3 [0.57636505]
4 [0.6375247]
5 [0.6089247]
6 [0.5801759]
7 [0.65746486]
8 [0.5248315]
9 [0.6150158]
10 [0.7627026]
11 [0.6789244]
12 [0.62114394]
13 [0.70664436]
14 [0.6750711]
15 [0.7513515]
16 [0.688979]
17 [0.53158844]
18 [0.5929505]
19 [0.65841246]
20 [0.76393914]
21 [0.49811614]
22 [0.5720482]
23 [0.65741014]
24 [0.65178907]
25 [0.765481]
26 [0.74699557]
27 [0.73477656]
28 [0.5688213]
29 [0.5649129]
30 [0.49355912]
31 [0.73064524]
32 [0.5279353]
33 [0.45890772]
34 [0.7300871]
35 [0.6584046]
36 [0.7264738]
37 [0.7818107]
38 [0.6903262]
39 [0.6081654]
40 [0.6588137]
41 [0.65812033]
42 [0.8004648]
43 [0.5476092]
44 [0.6686086]
45 [0.5684219]
46 [0.73655653]
47 [0.4729843]
48 [0.52921236]
49 [0.7782579]
50 [0.7530558]
51 [0.5169469]
52 [0.50229394]
53 [0.6798171]
54 [0.7468913]
55 [0.7807407]
56 [0.7799375]
57 [0.7984641]
58 [0.6702175]
59 [0.64370525]
60 [0.5767253]
61 [0.5620408]
62 [0.514761]
63 [0.54828864]
64 [0.55576766]
65 [0.5736238]


In [69]:
def normalize2(sentence):
    '''
    Input Arguments:
        string of words
    return:
        string of words in standardize form
    '''
    sentence = re.sub("[إأآا]", "ا", sentence)
    sentence = re.sub("ى", "ي", sentence)
    sentence = re.sub("ؤ", "ء", sentence)
    sentence = re.sub("ئ", "ء", sentence)
    sentence = re.sub("ة", "ه", sentence)
    sentence = re.sub("گ", "ك", sentence)
    return sentence

In [ ]:
def removing_ar_stopwords(text):
    """
        Removes Stop Words
        
    """

    original_words = []
    words = word_tokenize(text) # works on one string not list
    for word in words:
        if word not in stop_words:
            original_words.append(word)
    filtered_sentence = " ".join(original_words)
    return filtered_sentence

In [ ]:
def clearReg(text):
    """
        This function for getting the normal values of out of lemmatization function
        that takse a string of dict as a 
        takes  : '{"result":["امر","ب","أخذ","ما","نهى","ه","انتهى"]}'
        return : ['امر أخذ ما نهى انتهى']
    """
    each_lemma_word = []
    each_lemma_sentence = []
    for hadith in text:
        matches = re.findall(r'\"(.+?)\"',hadith)
        for word in matches:
            if len(word) >= 2 and word !='result':
                each_lemma_word.append(word)
        each_lemma_sentence.append(" ".join(each_lemma_word))
        each_lemma_word.clear()
    return each_lemma_sentence

In [ ]:
def stemming_1(text):
    """
        1st Stemming function - NLTK by ISRIStemmer.
    """
    st = ISRIStemmer()
    stemmend_words = []
    words = word_tokenize(text)
    for word in words:
        stemmend_words.append(st.stem(word))
    stemmed_sentence = " ".join(stemmend_words)
    return stemmed_sentence
        
    
    
def stemming_2(text):
    """
        2nd Stemming Function with built in library called Tashaphyne.
        The documentation ==> https://pypi.org/project/Tashaphyne/
    
    """
    import pyarabic.arabrepr
    arepr = pyarabic.arabrepr.ArabicRepr()
    repr = arepr.repr

    from tashaphyne.stemming import ArabicLightStemmer
    ArListem = ArabicLightStemmer()

    hadiths_without_stop_words_and_with_normalization_and_with_stemming = []

    for hadith in hadiths_without_stop_words_and_with_normalization:
        words = word_tokenize(hadith)
        new_list = []
        for word in words:
            stem = ArListem.light_stem(word)
            stem = ArListem.get_stem()
            new_list.append(stem)

        hadith_sentence_with_stemming = " ".join(new_list)
        hadiths_without_stop_words_and_with_normalization_and_with_stemming.append(hadith_sentence_with_stemming)
        
    return hadiths_without_stop_words_and_with_normalization_and_with_stemming

In [ ]:
def lemmatization(text):
    """
        1st Lemmatizing Function using Farasa API, and it should get best result of all the previous functions
        return a string from the dictionary corpus '{"result":["امر","ب","أخذ","ما","نهى","ه","انتهى"]}'
    """
    import http.client
    conn = http.client.HTTPSConnection("farasa-api.qcri.org")
    hadith_dict = {}
    list_pyload_input = []
    list_pyload_out = []
    length = len(text)
    for h in text[:length]:
        q = '{"text":'+'"{}"'.format(h)+'}'
        list_pyload_input.append(q)
    headers = { 'content-type': "application/json", 'cache-control': "no-cache", }
    for h in list_pyload_input:
        conn.request("POST", "/msa/webapi/lemma", h.encode('utf-8'), headers)
        res = conn.getresponse()
        data = res.read()
        list_pyload_out.append(data.decode("utf-8"))
        final_result = clearReg(list_pyload_out)     # call clearReg for clean the text
    return final_result

In [ ]:
# Sahih_Bukhari
cleared_Hadith_1 = []           # Removing stopwords
cleared_Hadith_1_2 = []         # Normalization
cleared_Hadith_1_2_3 = []       # Lematization

for hadith in all_hadiths_1:
    cleared_Hadith_1.append(removing_ar_stopwords(hadith))         # Removing stopwords
for hadith in cleared_Hadith_1:
    cleared_Hadith_1_2.append(normalize(hadith))                   # Normalization
cleared_Hadith_1_2_3 = lemmatization(cleared_Hadith_1_2)           # Lematization

print('The size of data:')
len(cleared_Hadith_1), len(cleared_Hadith_1_2), len(cleared_Hadith_1_2_3)

In [ ]:
cleared_Hadith_1_2_3[:1]
